### Data Generator

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [4]:
train_idg = image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True
)

val_idg = image.ImageDataGenerator(
    rescale=1./255
)

In [5]:
train_generator = train_idg.flow_from_directory(
    '../Dataset/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse'
)

Found 273 images belonging to 2 classes.


In [6]:
val_generator = val_idg.flow_from_directory(
    '../Dataset/test',
    target_size=(224,224),
    batch_size=24,
    class_mode='sparse'
)

Found 68 images belonging to 2 classes.


### Scratch Model

In [7]:
from keras.layers import *
from keras.models import Sequential

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [11]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

- Train Model

In [12]:
from keras.callbacks import ModelCheckpoint
import os

In [13]:
if not os.path.exists('../Weights/Scratch'):
    os.mkdir('../Weights/Scratch')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [14]:
import os
import glob

In [15]:
files = glob.glob('../Weights/Scratch/*')
for f in files:
    os.remove(f)

In [16]:
checkpoint = ModelCheckpoint('../Weights/Scratch/model_{epoch}_{val_accuracy:.2f}.h5', 
                             monitor='val_accuracy', 
                             verbose=1, 
                             mode='auto',
                             #save_weights_only=True
                            )

In [17]:
hist = model.fit(
    train_generator,
    steps_per_epoch=9,
    epochs=10,
    validation_data=val_generator,
    validation_steps=3,
    callbacks=checkpoint
)

Epoch 1/10
9/9 [==============================] - ETA: 0s - loss: 0.6873 - accuracy: 0.5824
Epoch 00001: saving model to ../Weights/Scratch\model_1_0.54.h5
9/9 [==============================] - 78s 9s/step - loss: 0.6873 - accuracy: 0.5824 - val_loss: 0.6926 - val_accuracy: 0.5441
Epoch 2/10
9/9 [==============================] - ETA: 0s - loss: 0.5822 - accuracy: 0.7326
Epoch 00002: saving model to ../Weights/Scratch\model_2_0.79.h5
9/9 [==============================] - 70s 8s/step - loss: 0.5822 - accuracy: 0.7326 - val_loss: 0.5102 - val_accuracy: 0.7941
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.4653 - accuracy: 0.8462
Epoch 00003: saving model to ../Weights/Scratch\model_3_0.94.h5
9/9 [==============================] - 66s 7s/step - loss: 0.4653 - accuracy: 0.8462 - val_loss: 0.5404 - val_accuracy: 0.9412
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.3969 - accuracy: 0.9158
Epoch 00004: saving model to ../Weights/Scratch\model_4_0.5

In [18]:
x,y=val_generator.next()

In [19]:
model.evaluate(x,y)

1/1 [==============================] - 0s 3ms/step - loss: 0.3143 - accuracy: 1.0000


[0.31434568762779236, 1.0]

- Save Model History

In [20]:
import numpy as np
import os

In [21]:
if not os.path.exists('../Model_history'):
    os.mkdir('../Model_history')
    print('Directory created')
else:
    print('Already Created')

Already Created


In [22]:
np.save('../Model_history/scratch_file.npy',hist.history)
read_dictionary = np.load('../Model_history/scratch_file.npy',allow_pickle='TRUE').item()
print(read_dictionary.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
